# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**:Fernando Ramos

**Professor**: Pablo Camarillo Ramirez

In [22]:
import findspark
findspark.init()

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, trim, col, count, isnull, when, \
    lit, concat, round, asc, desc, get_json_object
from datetime import datetime
from fernandoramos.spark_utils import SparkUtils

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://d3eb0343c341:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [24]:
schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])

df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")

In [25]:
df_agencies = df_agencies.withColumn("agency_name", get_json_object(df_agencies.agency_info, "$.agency_name"))

In [26]:
schema_agencies = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

df_rental = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")

In [27]:
##Extract agency_id
df_rental = df_rental.withColumn("agency_id", get_json_object(df_rental.rental_info, "$.agency_id")) \
                     .withColumn("car_id", get_json_object(df_rental.rental_info, "$.car_id")) \
                     .withColumn("customer_id", get_json_object(df_rental.rental_info, "$.customer_id"))

In [34]:
schema_agencies = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])

df_vehicle = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")

In [36]:
df_vehicle = df_vehicle.withColumn("car_name", get_json_object(df_vehicle.car_info, "$.car_name"))

In [37]:
schema_agencies = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])

df_customer = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_customer = df_customer.withColumn("customer_name", get_json_object(df_customer.customer_info, "$.customer_name"))

In [38]:
customer_to_join = df_customer.select("customer_id", "customer_name")
df_final = df_rental.join(customer_to_join, on="customer_id")

car_to_join = df_car.select("car_id", "car_name")
df_final = df_final.join(car_to_join, on="car_id")

agency_to_join = df_agencies.select("agency_id", "agency_name")
df_final = df_final.join(agency_to_join, on="agency_id")

In [39]:
df_show = df_final.select("rental_id", "customer_name", "car_name", "agency_name")

In [46]:
df_show.show(truncate=False)

+---------+---------------+-----------------------------------+-------------+
|rental_id|customer_name  |car_name                           |agency_name  |
+---------+---------------+-----------------------------------+-------------+
|11891    |Margaret Jones |Wallace-Carlson Model 9            |NYC Rentals  |
|11892    |Albert Williams|Grimes-Green Model 8               |LA Car Rental|
|11893    |Caleb Fleming  |Stewart-Allen Model 5              |SF Cars      |
|11894    |Andrew Butler  |Campos PLC Model 4                 |NYC Rentals  |
|11895    |Kristin Potts  |Wagner LLC Model 1                 |SF Cars      |
|11896    |Jeremy Parks   |Jones, Jefferson and Rivera Model 7|LA Car Rental|
|11897    |Terry Wells    |Lopez and Sons Model 9             |Zapopan Auto |
|11898    |Marc Williams  |Salazar Ltd Model 8                |SF Cars      |
|11899    |Danny Williams |Villanueva PLC Model 7             |LA Car Rental|
|11900    |Eric Owens PhD |Faulkner-Howard Model 5            |S

In [52]:
df_show.write \
    .partitionBy("agency_name") \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/opt/spark/work-dir/data/car_service/output/csv")

In [ ]:
sc.stop()